# Load config

In [1]:
import yaml
with open("../config.yaml") as f:
    config = yaml.safe_load(f)

# Alpaca py sample

Data only available from December 2015 onwards

In [2]:
import datetime

from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame

    

client = StockHistoricalDataClient(config['alpaca']['key'], config['alpaca']['secret'])

assets = ["SPY", "IEF", "TLT", "GLD", "VNQ"]

request_params = StockBarsRequest(
                        symbol_or_symbols=assets,
                        timeframe=TimeFrame.Day,
                        start=datetime.datetime(2012, 7, 1)
                 )

bars = client.get_stock_bars(request_params).df
bars

open     high      low   close  \
symbol timestamp                                                       
GLD    2015-12-01 05:00:00+00:00  102.3000  102.430  101.810  102.28   
       2015-12-02 05:00:00+00:00  101.3500  101.350  100.530  100.69   
       2015-12-03 05:00:00+00:00  101.2300  101.970  100.845  101.76   
       2015-12-04 05:00:00+00:00  102.7400  104.210  102.650  104.02   
       2015-12-07 05:00:00+00:00  103.3200  103.370  102.500  102.67   
...                                    ...      ...      ...     ...   
VNQ    2023-02-27 05:00:00+00:00   86.5400   87.150   85.470   85.84   
       2023-02-28 05:00:00+00:00   85.8020   86.840   85.620   85.72   
       2023-03-01 05:00:00+00:00   85.1200   85.370   83.960   84.64   
       2023-03-02 05:00:00+00:00   84.0400   85.760   83.920   85.63   
       2023-03-03 05:00:00+00:00   86.3099   87.125   86.060   86.93   

                                      volume  trade_count        vwap  
symbol timestamp                                                       
GLD    2015-12-01 05:00:00+00:00   4645916.0      19450.0  102.161847  
       2015-12-02 05:00:00+00:00  10223492.0      42784.0  100.947089  
       2015-12-03 05:00:00+00:00   6499935.0      30659.0  101.426721  
       2015-12-04 05:00:00+00:00  10670249.0      48021.0  103.717041  
       2015-12-07 05:00:00+00:00   4393461.0      23083.0  102.879428  
...                                      ...          ...         ...  
VNQ    2023-02-27 05:00:00+00:00   5170988.0      58570.0   86.112348  
       2023-02-28 05:00:00+00:00   7262105.0      65269.0   86.218755  
       2023-03-01 05:00:00+00:00   7042052.0      76889.0   84.524911  
       2023-03-02 05:00:00+00:00   5208876.0      56857.0   84.902620  
       2023-03-03 05:00:00+00:00   4542142.0      52913.0   86.748589  

[9130 rows x 7 columns]

# cTrader Open API sample

In [3]:
def transformTrendbar(trendbar):
    openTime = datetime.datetime.fromtimestamp(trendbar.utcTimestampInMinutes * 60, datetime.timezone.utc)
    openPrice = (trendbar.low + trendbar.deltaOpen) / 100000.0
    highPrice = (trendbar.low + trendbar.deltaHigh) / 100000.0
    lowPrice = trendbar.low / 100000.0
    closePrice = (trendbar.low + trendbar.deltaClose) / 100000.0
    return [openTime, openPrice, highPrice, lowPrice, closePrice, trendbar.volume]

symbolName = "EURUSD"

In [4]:
from ctrader_open_api import Client, Protobuf, TcpProtocol, Auth, EndPoints
from ctrader_open_api.messages.OpenApiCommonMessages_pb2 import *
from ctrader_open_api.messages.OpenApiMessages_pb2 import *
from ctrader_open_api.messages.OpenApiModelMessages_pb2 import *
from twisted.internet import reactor
import json
import datetime
import calendar

host = EndPoints.PROTOBUF_LIVE_HOST if config["ctrader"]["hostType"].lower() == "live" else EndPoints.PROTOBUF_DEMO_HOST
client = Client(host, EndPoints.PROTOBUF_PORT, TcpProtocol)

dailyBars = []

def onError(failure): # Call back for errors
    print("Message Error: ", failure)

def disconnected(client, reason): # Callback for client disconnection
    print("\nDisconnected: ", reason)

def onMessageReceived(client, message): # Callback for receiving all messages
    print("Message received: \n", Protobuf.extract(message))

def connected(client): # Callback for client connection
    print("\nConnected")
    # Now we send a ProtoOAApplicationAuthReq
    request = ProtoOAApplicationAuthReq()
    request.clientId = config["ctrader"]["id"]
    request.clientSecret = config["ctrader"]["secret"]
    # Client send method returns a Twisted deferred
    deferred = client.send(request)
    # You can use the returned Twisted deferred to attach callbacks
    # for getting message response or error backs for getting error if something went wrong
    deferred.addCallbacks(applicationAuthResponseCallback, onError)
    deferred.addErrback(onError)

def applicationAuthResponseCallback(result):
    print("\nApplication authenticated")
    request = ProtoOAAccountAuthReq()
    request.ctidTraderAccountId = config["ctrader"]["accountId"]
    request.accessToken = config["ctrader"]["accessToken"]
    deferred = client.send(request)
    deferred.addCallbacks(accountAuthResponseCallback, onError)

def accountAuthResponseCallback(result):
    print("\nAccount authenticated")
    request = ProtoOASymbolsListReq()
    request.ctidTraderAccountId = config["ctrader"]["accountId"]
    request.includeArchivedSymbols = False
    deferred = client.send(request)
    deferred.addCallbacks(symbolsResponseCallback, onError)

def symbolsResponseCallback(result):
    print("\nSymbols received")
    symbols = Protobuf.extract(result)
    symbolsFilterResult = list(filter(lambda symbol: symbol.symbolName == symbolName, symbols.symbol))
    if len(symbolsFilterResult) == 0:
        raise Exception(f"There is symbol that matches to your defined symbol name: {symbolName}")
    elif len(symbolsFilterResult) > 1:
        raise Exception(f"More than one symbol matched with your defined symbol name: {symbolName}, match result: {symbolsFilterResult}")
    symbol = symbolsFilterResult[0]
    request = ProtoOAGetTrendbarsReq()
    request.symbolId = symbol.symbolId
    request.ctidTraderAccountId = config["ctrader"]["accountId"]
    request.period = ProtoOATrendbarPeriod.D1
    # We set the from/to time stamps to 50 weeks, you can load more data by sending multiple requests
    # Please check the ProtoOAGetTrendbarsReq documentation for more detail
    request.fromTimestamp = int(calendar.timegm((datetime.datetime.utcnow() - datetime.timedelta(weeks=50)).utctimetuple())) * 1000
    request.toTimestamp = int(calendar.timegm(datetime.datetime.utcnow().utctimetuple())) * 1000
    deferred = client.send(request)
    deferred.addCallbacks(trendbarsResponseCallback, onError)

def trendbarsResponseCallback(result):
    print("\nTrendbars received")
    trendbars = Protobuf.extract(result)
    barsData = list(map(transformTrendbar, trendbars.trendbar))
    global dailyBars
    dailyBars.clear()
    dailyBars.extend(barsData)
    print("\ndailyBars length:", len(dailyBars))
    print("\Stopping reactor...")
    reactor.stop()


# Setting optional client callbacks
client.setConnectedCallback(connected)
client.setDisconnectedCallback(disconnected)
client.setMessageReceivedCallback(onMessageReceived)

In [5]:
# Starting the client service
client.startService()
# Run Twisted reactor
reactor.run()


Connected
Message received: 
 

Application authenticated
Message received: 
 ctidTraderAccountId: 24136091


Account authenticated
Message received: 
 ctidTraderAccountId: 24136091
symbol {
  symbolId: 10020
  symbolName: "XNGUSD"
  enabled: true
  baseAssetId: 45
  quoteAssetId: 16
  symbolCategoryId: 7
  description: "Natural Gas"
}
symbol {
  symbolId: 44
  symbolName: "XAGEUR"
  enabled: true
  baseAssetId: 17
  quoteAssetId: 5
  symbolCategoryId: 2
  description: "Silver vs Euro"
}
symbol {
  symbolId: 42
  symbolName: "XAGUSD"
  enabled: true
  baseAssetId: 17
  quoteAssetId: 16
  symbolCategoryId: 2
  description: "Silver vs US Dollar"
}
symbol {
  symbolId: 43
  symbolName: "XAUEUR"
  enabled: true
  baseAssetId: 18
  quoteAssetId: 5
  symbolCategoryId: 2
  description: "Gold vs Euro"
}
symbol {
  symbolId: 41
  symbolName: "XAUUSD"
  enabled: true
  baseAssetId: 18
  quoteAssetId: 16
  symbolCategoryId: 2
  description: "Gold vs US Dollar"
}
symbol {
  symbolId: 10016
  symb

In [6]:
import pandas as pd

pd.DataFrame(dailyBars).rename(columns={0: "timestamp", 1: "open", 2: "high", 3: "low", 4: "close", 5: "volume"}).set_index("timestamp")

,open,high,low,close,volume
timestamp,,,,,
2022-03-20 21:00:00+00:00,1.10574,1.10701,1.10103,1.10168,118582
2022-03-21 21:00:00+00:00,1.10153,1.10458,1.09610,1.10246,136869
2022-03-22 21:00:00+00:00,1.10238,1.10441,1.09640,1.10057,117124
2022-03-23 21:00:00+00:00,1.10056,1.10140,1.09660,1.09970,125945
2022-03-24 21:00:00+00:00,1.09941,1.10378,1.09810,1.09835,129440
...,...,...,...,...,...
2023-02-26 22:00:00+00:00,1.05439,1.06198,1.05329,1.06088,120589
2023-02-27 22:00:00+00:00,1.06042,1.06453,1.05739,1.05755,123716
2023-02-28 22:00:00+00:00,1.05755,1.06914,1.05653,1.06690,152483
